In [ ]:
# Sets up plotting
%matplotlib notebook

## Numerical Approximation Methods
---
### Euler's Method

Euler's method is an algorithm for approximating solutions to first-order IVPs

$$
\frac{dy}{dt} = f(t,y) \quad y(t_0) = y_0.
$$

The key idea of Euler's method and all numerical techniques is
**discretization**. If a variable can take on any value between two
specified values, it is called a **continuous** variable;
otherwise, it is called a **discrete** variable.

When the independent variable is time, $t$, we **discretize** it by
imagining it to correspond with the ticks of a clock, just like how
the second hand on a clock moves jerkily as opposed to smoothly from
one second to the next.

Our discrete ticks are determined by a **start time**, $t_0$, and a
tick size, or **step size** $h = \Delta t$, for example $t_0,
\underbrace{t_0+h}_{t_1}, \underbrace{t_0+2h}_{t_2},
\underbrace{t_0+3h}_{t_3}, \ldots$


The next step is to pick a starting point or **initial condition**:
$(t_0, y_0)$, then we use $f(t,y)$ to generate a list of coordinate
pairs which can be plotted, i.e. $(t_0, y_0), (t_1,y_1), (t_2,y_2),
(t_3,y_3), \ldots$


The above list of pairs is generated via the following **recursive** algorithm.

$$
\begin{align}
  t_{i+1} &= t_i + h \\
  y_{i+1} &= y_i + \underbrace{h \cdot f(t_i, y_i)}_{\Delta y}
\end{align}
$$

_Note:_ You will need to define a Python function, `f(t,y)`, before you can call `euler`. This `f(t,y)` function is the right hand side of the differential equation $y' = f(t,y)$.

In [ ]:
def euler(t0, y0, h, n):

    t = [0.0]*(n+1) # n+1 t values
    y = [0.0]*(n+1) # n+1 y values

    t[0] = t0
    y[0] = y0

    # main loop which fills both lists
    for i in range(n):
        t[i+1] = t[i] + h
        y[i+1] = y[i] + h*f(t[i],y[i])
        
    return t,y

In [ ]:
def f(t,y):
    return y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

T = np.linspace(0, 2, 100)
te,ye = euler(0,1,0.5,4)

plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(T, np.exp(T), label='Exact', color='green')
plt.plot(te, ye, label='Euler', color='blue',marker='.')
plt.xlabel('t')
plt.ylabel('y')
plt.legend()
#plt.savefig('Euler.pdf')
plt.show()

### Heun (Improved Euler)

The idea of the Heun method is to compute the slope of the tangent line to the solution curve at both endpoints of a single time step, i.e. at $t_i$ and $t_i + h$, and then average these two slopes to get a better approximation of the true (exact) solution curve.

The algorithm generates a pair of lists. The main loop of the algorithm is similar to Euler's method, except now we compute two slopes and average them in order to compute the new $y$ value, i.e. $y_{i+1}$. In the pseudo-code below, $k_1$ and $k_2$ are the slopes of tangent lines to the solution curve at the two  endpoints of the interval $(t_i, t_i + h)$.

$$
\begin{align}
  t_{i+1} &= t_i + h \\
  k_1 &= f(t_i, y_i) \\
  k_2 &= f(t_i + h, y_i + h \cdot k_1) \\
  y_{i+1} &= y_i + \underbrace{h \cdot \tfrac{1}{2} \cdot (k_1 + k_2)}_{\Delta y}
\end{align}
$$

In [ ]:
def heun(t0, y0, h, n):
    t = [0.0]*(n+1)
    y = [0.0]*(n+1)
    t[0] = t0
    y[0] = y0
    
    for i in range(n):
        t[i+1] = t[i] + h
        k1 = f(t[i], y[i])
        k2 = f(t[i]+h, y[i]+h*k1)
        y[i+1] = y[i] + h*0.5*(k1+k2)

    return t,y

In [ ]:
th, yh = heun(0, 1, 0.5, 4)
te,ye = euler(0,1,0.5,4)

plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(T, np.exp(T), label='Exact', color='green')
plt.plot(te, ye, label='Euler', color='blue',marker='.')
plt.plot(th, yh, label='Heun', color='orange',marker='.')
plt.xlabel('t')
plt.ylabel('y')
plt.legend()
#plt.savefig('Euler_Heun.pdf')
plt.show()


### Runge-Kutta

The core idea of the Runge-Kutta method is to do Heun's method twice. To do this we take two steps of time duration $h/2$. Doing Heun's method twice requires us to generate four slopes, $k_1, k_2, k_3$ and $k_4$. After generating these four slopes we do a special _weighted average_ where we emphasize the contribution of the slopes at the middle of the time interval, $k_2$ and $k_3$ and de-emphasize the contribution of the slopes computed at the two edges of the interval. The weighted average is computed via: $\tfrac{1}{6}\cdot(k_1 + 2k_2 + 2k_3 + k_4)$.

$$
\begin{align}
  t_{i+1} &= t_i + h \\
  k_1 &= f(t_i, y_i) \\
  k_2 &= f(t_i + \tfrac{1}{2} \cdot h, y_i + \tfrac{1}{2} \cdot h \cdot k_1) \\
  k_3 &= f(t_i + \tfrac{1}{2} \cdot h, y_i + \tfrac{1}{2} \cdot h \cdot k_2) \\
  k_4 &= f(t_i + h, y_i + h \cdot k_3) \\
  y_{i+1} &= y_i + \underbrace{h \cdot \tfrac{1}{6} \cdot (k_1 + 2k_2 + 2k_3 + k_4)}_{\Delta y}
\end{align}
$$

In [ ]:
def runge_kutta(t0, y0, h, n):
    t = [0.0]*(n+1)
    y = [0.0]*(n+1)
    t[0] = t0
    y[0] = y0
    
    for i in range(n):
        t[i+1] = t[i] + h
        k1 = f(t[i], y[i])
        k2 = f(t[i]+ 0.5*h, y[i] + 0.5*h*k1)
        k3 = f(t[i]+ 0.5*h, y[i] + 0.5*h*k2)
        k4 = f(t[i]+ h, y[i] + h*k3)
        y[i+1] = y[i] + h*(1/6)*(k1 + 2*k2 + 2*k3 + k4)

    return t,y

In [ ]:
tr, yr = runge_kutta(0, 1, 0.5, 4)

plt.figure(figsize=(10, 8), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(T, np.exp(T), label='Exact', color='green')
plt.plot(te, ye, label='Euler', color='blue',marker='.')
plt.plot(th, yh, label='Heun', color='orange',marker='.')
plt.plot(tr, yr, label='Runge-Kutta', color='red',marker='.')
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
#plt.savefig('Euler_Heun_RK.pdf')
plt.show()
